<a href="https://colab.research.google.com/github/ShubhamKumarNigam/ML-Practice-Code/blob/master/Yelp_classification_using_Simple_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reference [Simple Transformers — Introducing The Easiest Way To Use BERT, RoBERTa, XLNet, and XLM](https://towardsdatascience.com/simple-transformers-introducing-the-easiest-bert-roberta-xlnet-and-xlm-library-58bf8c59b2a3)


In [1]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 8.0MB/s 
     |████████████████████████████████| 1.1MB 16.1MB/s 
     |████████████████████████████████| 890kB 28.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=69482565880bf58a8385eae503bc052fa9820322546a3d597199dca209b9eb91
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 8.3MB/s 
     |████████████████████████████████| 3.0MB 15.0MB/s 
     |████████████████████████████████| 317kB 58.9MB/s 
     |████████████████████████████████| 890kB 49.4MB/s 
     |████████████████████████████████| 1.6MB 55.8MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 7.2MB 65.0MB/s 
     |████████████████████████████████| 102kB 16.8MB/s 
     |████████████████████████████████| 122kB 60.3MB/s 
     |████████████████████████████████| 163kB 62.7MB/s 
     |████████████████████████████████| 102kB 15.8MB/s 
     |████████████████████████████████| 4.4MB 51.7MB/s 
     |████████████████████████████████| 112kB 62.4MB/s 
     |████████████████████████████████| 71kB 12.2MB/s 
     |████████████████████████████████| 122kB 56.2MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7423 sha256=1ad0bac5f6f21b3dd8b14e0cdcb0d420b22fe954dccb621b3566e5cace7d8c49
  Stored i

In [3]:
!pip install tensorboardx

In [4]:
!mkdir data
!wget https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz -O data/data.tgz

--2020-09-13 16:07:20--  https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.44.110
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.44.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166373201 (159M) [application/x-tar]
Saving to: ‘data/data.tgz’

data/data.tgz       100%[===================>] 158.67M  35.0MB/s    in 5.1s    

2020-09-13 16:07:26 (30.9 MB/s) - ‘data/data.tgz’ saved [166373201/166373201]



In [5]:

!tar -xvzf data/data.tgz -C data/
!mv data/yelp_review_polarity_csv/* data/
!rm -r data/yelp_review_polarity_csv/
!rm data/data.tgz

yelp_review_polarity_csv/
yelp_review_polarity_csv/train.csv
yelp_review_polarity_csv/readme.txt
yelp_review_polarity_csv/test.csv


In [6]:
import pandas as pd


prefix = 'data/'

train_df = pd.read_csv(prefix + 'train.csv', header=None)
train_df.head()


,0,1
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [7]:
eval_df = pd.read_csv(prefix + 'test.csv', header=None)
print(eval_df.head())

   0                                                  1
0  2  Contrary to other reviews, I have zero complai...
1  1  Last summer I had an appointment to get new ti...
2  2  Friendly staff, same starbucks fair you get an...
3  1  The food is good. Unfortunately the service is...
4  2  Even when we didn't have a car Filene's Baseme...


In [8]:
train_df[0] = (train_df[0] == 2).astype(int)
eval_df[0] = (eval_df[0] == 2).astype(int)

train_df = pd.DataFrame({
    'text': train_df[1].replace(r'\n', ' ', regex=True),
    'label':train_df[0]
})

print(train_df.head())

                                                text  label
0  Unfortunately, the frustration of being Dr. Go...      0
1  Been going to Dr. Goldberg for over 10 years. ...      1
2  I don't know what Dr. Goldberg was like before...      0
3  I'm writing this review to give you a heads up...      0
4  All the food is great here. But the best thing...      1


In [9]:
eval_df = pd.DataFrame({
    'text': eval_df[1].replace(r'\n', ' ', regex=True),
    'label':eval_df[0]
})

print(eval_df.head())

                                                text  label
0  Contrary to other reviews, I have zero complai...      1
1  Last summer I had an appointment to get new ti...      0
2  Friendly staff, same starbucks fair you get an...      1
3  The food is good. Unfortunately the service is...      0
4  Even when we didn't have a car Filene's Baseme...      1


In [10]:
print("Length of Traing Data",len(train_df))

sample_train_df = train_df[:5000]

print("Length of Sampled Traing Data", len(sample_train_df))

print("Length of Evaluation Data", len(eval_df))

sample_eval_df = eval_df[:500]

print("Length of Sampled Evaluation Data", len(sample_eval_df))

Length of Traing Data 560000
Length of Sampled Traing Data 5000
Length of Evaluation Data 38000
Length of Sampled Evaluation Data 500


In [ ]:
from simpletransformers.classification import ClassificationModel


# Create a TransformerModel
model = ClassificationModel('roberta', 'roberta-base')

# Train the model
model.train_model(sample_train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(sample_eval_df)


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:304: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
predictions, raw_outputs = model.predict(['Friendly staff, same starbucks fair you get anywhere else.  Sometimes the lines can get long.'])

In [11]:
from simpletransformers.classification import ClassificationModel


# Create a TransformerModel
# model = ClassificationModel('roberta', 'roberta-base')
model = ClassificationModel('bert', 'bert-base-cased', args={'num_train_epochs': 3, 'learning_rate': 3e-5, 'overwrite_output_dir': False,})

# Train the model
model.train_model(sample_train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(sample_eval_df)


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:304: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:776: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


NameError: ignored

In [12]:
predictions, raw_outputs = model.predict(['Friendly staff, same starbucks fair you get anywhere else.  Sometimes the lines can get long.'])

In [14]:
print(predictions, raw_outputs)

[1] [[-3.3260972  3.3314672]]
